In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%matplotlib qt

## Playing around with realistic RFI signals

These suite of functions create RFI signals with various digital encoding schemes.

<br>

Most of them follow the same basic flow:

\> Create random bit sequence

\> Create symbol sequence from bit sequence , maybe smooth it

\> Apply to carrier frequency signal

\> Return time-domain signal and symbol sequence

<br>

Remaining issues:

RFI signals are way too wide in frequency space - need to occupy two or less channels

Input power is always the same (except for amplitude-encoded signals), no code to change the power is written

Frequency-shift encoding functions do not maintain phase across frequency shifts

# Warning: the signal functions below are copied in ../signal.py - those are meant to be more complete versions, while this notebook is intended as a scratch space


In [3]:
#create square wave (for duty cycle)
def sq(x,dc,dc_per):
        y = np.zeros(len(x))
        for i in range(len(x)//dc_per):
            start = int(i*dc_per)
            end = int(start + dc*dc_per)
            #y[start:end] = 1
            y[start:end] = np.hamming(end-start)
        return y


In [2]:
#Simulating r0800x4096 observations
bw = 800e6
fs = bw
ts = 1/fs



Nchan = 4096
Nint = 1
N=Nchan*Nint
x = np.arange(N)

#frequency axis
m = x*fs/N

## Guide to RFI waveform function inputs

=======================================

x       : index array (np.arange(N)) for N total points

dB      : power of input signal. I don't even have the code written yet, so keep this at 0.

fc      : carrier frequency of RFI signal.

f0\*     : space frequency (bit = 0) for frequency shift-keyed encoding

f1\*     : mark frequency (bit = 1) for frequency shift-keyed encoding

T_bit\*  : number of time samples per encoded bit.

bias    : relative voltage bias for amplitude-encoded signals. So that bit = 00 isn't zero voltage.

======================================

*fc replaced by f0/f1 in frequency-shift encoding


*At sampling frequency of 800MHz, T_bit=400 is a 2Mbit/s data rate



In [3]:
#===========================================
#Phase-shift encoding
#===========================================

def bpsk(x,dB,fc,T_bit):
    bit_seq = np.random.randint(0,high=2,size=int(len(x)/T_bit)+1)
    sym_seq = 2*bit_seq-1
    
    p_of_t = np.ones(T_bit)
    s_of_t = np.kron(sym_seq,p_of_t)[:len(x)]
    #apply carrier freq

    e_vec = np.exp(2.j*np.pi*fc*x*ts)
    x_of_t = s_of_t*e_vec
    return x_of_t,s_of_t
    

#quadrature phase shift keying
#same as binary phase shift, but information encoded is 2-bit, leading to 4 phases (45deg,135,225,315)
def qpsk(x,dB,fc,T_bit):
    sym_seq = np.random.randint(1,high=5,size = int(len(x)/T_bit)+1)
    pulse = np.ones(T_bit)
    sym_seq = np.kron(sym_seq,pulse)[:len(x)]
    
    arg = 1.j*((2*np.pi*fc*x*ts) + (2*sym_seq-1)*(np.pi/4))
    sig = np.exp(arg)
    
    return sig,sym_seq


In [4]:
#===========================================
#Frequency-shift encoding
#===========================================


#binary freq-shift keying - switch between 2 freqs
def bfsk(x,dB,f0,f1,T_bit):
    #make bit sequence
    bit_seq = np.random.randint(0,high=2,size=int(len(x)/T_bit))
    pulse = np.ones(T_bit)
    sym_seq = np.kron(bit_seq,pulse)
    
    #apply to mark/space frequencies
    #need to make sure phase doesn't change
    #there is a vectorized way to do this im sure but im lazy
    sig = np.zeros(len(x),dtype=np.complex64)
    phase = 0
    for i in range(int(len(x)/T_bit)):
        if bit_seq[i] == 1:
            arg = 1.j*((2*np.pi*fm*x[i*T_bit:(i+1)*T_bit]*ts)+phase)
            sig[i*T_bit:(i+1)*T_bit] = np.exp(arg)
            phase += fm*ts*T_bit
        else:
            arg = 1.j*((2*np.pi*fs*x[i*T_bit:(i+1)*T_bit]*ts)+phase)
            sig[i*T_bit:(i+1)*T_bit] = np.exp(arg)
            phase += fm*ts*T_bit

    return sig,sym_seq


#binary freq-shift keying - switch between 2 freqs, with smoothing of symbol sequence
def bfsk_smoothed(x,dB,fs,fm,T_bit):
    #make bit sequence
    bit_seq = np.random.randint(0,high=2,size=int(len(x)/T_bit)+1)
    pulse = np.ones(T_bit)
    sym_seq = np.kron(bit_seq,pulse)[:len(x)]

    gaus = np.exp((-(x-(len(x)/2))**2)/(2*(T_bit*0.1)**2))
    
    hann = np.hanning(int(T_bit*0.1))
    sym_seq = np.convolve(sym_seq,hann,mode='same')
    sym_seq = sym_seq/np.max(sym_seq)
    
    f_diff = fm-fs
    freq_seq = fs + (sym_seq)*f_diff

    arg = (2.j*np.pi*freq_seq*x*ts)
    sig = np.exp(arg)

    return sig,sym_seq,smoothed


In [5]:
#===========================================
#Amplitude-shift encoding
#===========================================

#4-level amplitude keying signal, with hanning smoothing
#bias: voltage bias
def ask_2bit(x,dB,fc,T_bit,bias):
    bit_seq = np.random.randint(0,5,size=int(len(x)/T_bit)+1)+bias
    #plt.plot(bit_seq)
    pulse = np.ones(T_bit)
    sym_seq = np.kron(bit_seq,pulse)[:len(x)]

    #smooth by hanning window that is 20% the time width of one bit
    hann1 = np.hanning(int(T_bit*0.2))
    #plt.plot(hann)
    sym_seq = np.convolve(sym_seq,hann1,mode='same')
    
    sym_seq = sym_seq / np.max(sym_seq)
    
    #apply carrier signal
    e_vec = np.exp(2.j*np.pi*fc*x*ts)
    
    sig = sym_seq * e_vec
    
    return sig,sym_seq

In [6]:
#perform FFT and determine frequency response

def freq_resp(sig,enc):
    N = len(sig)
    I = np.fft.fft(sig)
    I = I * I.conj()
    #I = np.fft.fftshift(I)
    I = I.real/(N)
    freq_vec = np.arange(0,800,800/4096)
    plt.plot(x,10*np.log10(I),label=enc)

In [11]:
#make some signals

x = np.arange(4096)

N0_dB = -10


#noise

n_of_t = np.random.normal(0,1,size=len(x)) + 1.j*np.random.normal(0,1,size=len(x))
noise_power = np.var(n_of_t)

N0_linear = 10**(N0_dB/10)
pow_factor = np.sqrt(N0_linear / noise_power)
n_of_t = n_of_t*pow_factor





In [49]:
sig,sym_seq = ask_2bit(x,0,400e6,400,0.5)

data = sig + n_of_t

freq_resp(data,'ask_2bit')

In [50]:
plt.legend()

In [51]:
plt.axhline(-10,linewidth=0.5)

In [52]:
plt.axhline(0,linewidth=0.5)
plt.axhline(10,linewidth=0.5)
plt.axhline(20,linewidth=0.5)
plt.axhline(30,linewidth=0.5)

In [53]:
plt.axvline(2048,linewidth=0.5)
plt.axvline(2028,linewidth=0.5)
plt.axvline(2008,linewidth=0.5)
plt.axvline(2068,linewidth=0.5)
plt.axvline(2088,linewidth=0.5)

In [60]:
x = np.arange(10000)
sine = np.sin(2*np.pi*x/2000)

y = sine * np.hanning(10000)


plt.plot(sine)
plt.plot(y)


In [62]:
plt.hist(sine,histtype='step',label='square window')
plt.hist(y,histtype='step',label='hanning window')
plt.legend()